## Motivation
Currently AI is advancing in the field of healthcare to improve detection of malignant tumors, give treatment recommendations, engage patients and support in administrative activities (Davenport and Kalakota 2019). Our goal is to contribute to this field by applying a neural network with transfer learning on a dataset with the aim to detect malignant cells of breast cancer. 

According to Krebsliga Schweiz (2021), there are 6’250 new cases and 1’410 deaths associated with breast cancer in Switzerland every year. Early diagnosis and treatment are a key to increasing the 5-year survival rate of patients.  

From a technical standpoint we want to investigate the performance differences between neural networks with and without transfer learning in the field of tumor detection.

## Data

We use the Kaggle dataset: Breast Histopathology Images, which contains 277’524 images that are classified whether the sample is positive or negative for Invasive Ductal Carcinoma (IDC). Therefore, we face a binary classification problem with this dataset. The sample dataset contains images scanned at 40x zoom that are prepared in 50 x 50-pixel patches.

[Kaggle Dataset](https://www.kaggle.com/paultimothymooney/breast-histopathology-images)

### Import Packages

In [1]:
import pandas as pd
import numpy as np
import datetime
import pickle

# Graphs, visualizations
import matplotlib.pyplot as plt

# import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, SGD

import sklearn
from sklearn.model_selection import train_test_split

In [2]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')


[]

### Load the Data

In [2]:
# Import Data From Pickle file

with open('y.pickle', 'rb') as f:
    y_data = pickle.load(f)
f.close()
y_data

with open('X.pickle', 'rb') as f:
    X_data = pickle.load(f)
f.close()

### Learnings
* The data of positive and negative samples is unbalanced, where patients have more negative patches than positive ones
* This could lead to an imbalanced result where we classify more patches as negative, which would be a severe mistake in cancer detection. A confusion matrix should be sufficient so verify this concern, when the model is trained

In [3]:
#Train-validation-test split
# train test split for validation after training x_test is never touched or looked at during training
x_train,x_test,y_train,y_test=sklearn.model_selection.train_test_split(np.asarray(X_data),
                                                                       np.asarray(y_data),
                                                                       test_size=.3,
                                                                       random_state=42)

# train test split for validation during training
x_train,x_val,y_train,y_val=sklearn.model_selection.train_test_split(x_train,
                                                                     y_train,
                                                                     test_size=.2,
                                                                     random_state=42)
# free the memory
del X_data
del y_data

#Dimension of the kaggle dataset
print((x_train.shape,y_train.shape))
print((x_val.shape,y_val.shape))
print((x_test.shape,y_test.shape))

((560, 50, 50, 3), (560,))
((140, 50, 50, 3), (140,))
((300, 50, 50, 3), (300,))


### Functions for plotting

In [5]:
def acc_plot(history):
    plt.plot(history.history['accuracy'], alpha=.6)
    plt.plot(history.history['val_accuracy'], alpha=.6)
    plt.xlabel('epoch')
    plt.legend(['train_acc', 'val_acc'], loc='upper left')
    return plt

def loss_plot(history):
    plt.plot(history.history['loss'][1:], alpha=.6)
    plt.plot(history.history['val_loss'], alpha=.6)
    plt.xlabel('epoch')
    plt.legend(['train_loss', 'val_loss'], loc='upper left')
    return plt

def plot_prc(name, labels, predictions, **kwargs):
    precision, recall, _ = sklearn.metrics.precision_recall_curve(labels, predictions)

    plt.plot(precision, recall, label=name, linewidth=2, **kwargs)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.grid(True)
    ax = plt.gca()
    ax.set_aspect('equal')
    
def conf_matrix(model, x_test, y_test):
    
    y_pred = [1 * (x[0]>=0.5) for x in model.predict(x_test)]

    matrix = sklearn.metrics.confusion_matrix(y_test, y_pred)
    df_cm = pandas.DataFrame(matrix, index = [i for i in ['No Cancer (actual)', 'Cancer (actual)']],
                      columns = [i for i in ['predict No Cancer', 'predict Cancer']])
    plt.figure(figsize = (10,7))
    sn.heatmap(df_cm, annot=True, fmt='d')
    return plt

### General model settings

Given the expermients conducted with dense layers, the geral settings for all transfer learning networks are as following:\
batch_size = 1024\
learning_rate = 0.001\
Adam Optimizer, earlie stopping and learning rate annealer, both with a patience of 20\
As the transfer architecture requires way more computation power, we will only train the network for maximally 50 epochs.

In [5]:
learningRate=0.001
batchSize = 1024
epochs = 50

#Early stopping callback
es = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=20, verbose=2,
                                      mode='max', baseline=None, restore_best_weights=True)
#Learning Rate Annealer
lrr = keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy',
                       factor=.1,
                       patience=20,
                       min_lr=1e-4,
                       verbose=2)
# Input shape
input_shape = (50, 50, 3)

# Transfer learning

For our transfer learning setting, we use Resnet50. Resnet50 is a deep convolutional network with, as the name states, 50 layers. Out of these 50 layers, 48 are convolutional layers, besides that it contains an average pooling layer and a maxpooling layer.\
It was trained with more than a million images from ImageNet (http://www.image-net.org/). Originally, the network was trained for multiclass classification with 1000 categories.\
The Keras API allows to load the Resnet50 with weights trained on image net. This is done with the keyword weights='imagenet'. Furthermore, Keras enables the user to define his own input shape. This is only possible if the original dense layer at the top of the network is not loaded. Therefore we use include_top=False. However, the number of channels must be 3 as the network was trained on colored images. Of course, we also freez the pretained weights, meaning that we don't update them during training.\
Firstly, we implement a simple network using resnet50. We call it simple, as we use just two denslayers to connect the Resnet50 with the desired output for our task. Before the Resnet50, we don't use any layer at all, but connect our input, images of the shape 50x50x3, directly to the Resnet50.\
In a next step, we try to enhance the model by adding layers before the Resnet50 and in a third step by adding a more complex architecture after Resnet50.

In this part of the project, we'd like to focus on the effect of the layers before and after the pretrained model, thus we don't conduct any experiments with different hyperparameters.

## Simple Transfer learning

In [7]:
model_input = keras.Input(shape=input_shape)

resnet = keras.applications.ResNet50(include_top=False,weights='imagenet',input_shape=(50,50,3))
resnet.trainable=False
x = resnet(model_input,training=False)

x = keras.layers.Flatten()(x)
x = keras.layers.Dense(1024, activation='relu')(x) # dense layer 1 
output = keras.layers.Dense(units=1, activation='sigmoid')(x)

In [8]:
model = keras.Model(inputs = model_input, outputs = output)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 50, 50, 3)]       0         
_________________________________________________________________
resnet50 (Functional)        (None, 2, 2, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 8192)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              8389632   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1025      
Total params: 31,978,369
Trainable params: 8,390,657
Non-trainable params: 23,587,712
_________________________________________________________________


In [14]:
model.compile(loss="binary_crossentropy", optimizer=Adam(epsilon=0.1, learning_rate=learningRate), metrics=["accuracy"])

history=model.fit(x_train, y_train,
                  batch_size=batchSize, epochs=epochs,
                  validation_data=(x_val,y_val),
                  callbacks=[es, lrr],
                 ) 

Epoch 1/50


KeyboardInterrupt: 

In [ ]:
# plot accuracy
plt = acc_plot(history)
plt.show()

In [ ]:
# plot loss
plt = loss_plot(history)
plt.show()

In [ ]:
plt = conf_matrix(model, x_test, y_test)
plt.show()

### Advanced Transfer Learning I

As mentioned, we try to improve the performance by changing the architecture before the Resnet50. For the simple model, we connected our input directly to the Resnet50. Now, we use UpSamling2D layers, these layers just double the input size but are not able to learn anything. Thus, for the advanced model, each UpSamling2D layer is followed by a Conv2D layer that maintains the shape. The intuition is that the Conv2D layer can learn something from the upsampled input before upsampling it further.

In [15]:
model_input = keras.Input(shape=input_shape)
beforeModel = keras.layers.UpSampling2D()(model_input)
beforeModel = keras.layers.Conv2D(3, 3, strides=1, padding="same")(beforeModel)
beforeModel = keras.layers.UpSampling2D()(beforeModel)
beforeModel = keras.layers.Conv2D(3, 3, strides=1, padding="same")(beforeModel)

resnet = keras.applications.ResNet50(include_top=False,weights='imagenet',input_shape=(200,200,3))
resnet.trainable=False
x = resnet(beforeModel,training=False)

x = keras.layers.Flatten()(x)
x = keras.layers.Dense(1024, activation='relu')(x) # dense layer 1 
output = keras.layers.Dense(units=1, activation='sigmoid')(x)

In [16]:
model = keras.Model(inputs = model_input, outputs = output)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 50, 50, 3)]       0         
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 100, 100, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 100, 100, 3)       84        
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 200, 200, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 200, 200, 3)       84        
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 100352)            0   

In [ ]:
model.compile(loss="binary_crossentropy", optimizer=Adam(epsilon=0.1, learning_rate=learningRate), metrics=["accuracy"])

history=model.fit(x_train, y_train,
                  batch_size=batchSize, epochs=epochs,
                  validation_data=(x_val,y_val),
                  callbacks=[es, lrr],
                 )  

Epoch 1/50


In [ ]:
# plot accuracy
plt = acc_plot(history)
plt.show()

In [ ]:
# plot loss
plt = loss_plot(history)
plt.show()

In [ ]:
plt = conf_matrix(model, x_test, y_test)
plt.show()

### Advanced Transfer Learning II

Finally, we attempt to improve the performance further by altering the architecture after the Resnet50. We introduce two types of regularization layers, namely BatchNormalization and Dropout. Besides that, we added an additional dense layer, so in total we use two dense layers and one output layer with sigmoid activation. After each dense layer, expect the output layer, we apply first a dropout layer with a dropout probability of 20% and then a batch normalization. After flattening the output of the resnet, we also apply a batch normalization. 

In [6]:
model_input = keras.Input(shape=input_shape)
beforeModel = keras.layers.UpSampling2D()(model_input)
beforeModel = keras.layers.Conv2D(3, 3, strides=1, padding="same")(beforeModel)
beforeModel = keras.layers.UpSampling2D()(beforeModel)
beforeModel = keras.layers.Conv2D(3, 3, strides=1, padding="same")(beforeModel)
resnet = keras.applications.ResNet50(include_top=False,weights='imagenet',input_shape=(200,200,3))
resnet.trainable=False
x = resnet(beforeModel,training=False)

x = keras.layers.Flatten()(x)
x = keras.layers.Dense(1024, activation='relu')(x) # dense layer 1 

x = keras.layers.Dropout(0.2)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Dense(512, activation='relu')(x) # dense layer 2
x = keras.layers.Dropout(0.2)(x)
output = keras.layers.Dense(units=1, activation='sigmoid')(x)

In [7]:
model = keras.Model(inputs = model_input, outputs = output)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 50, 50, 3)]       0         
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 100, 100, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 100, 100, 3)       84        
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 200, 200, 3)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 200, 200, 3)       84        
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten (Flatten)            (None, 100352)            0     

In [ ]:
model.compile(loss="binary_crossentropy", optimizer=Adam(epsilon=0.1, learning_rate=learningRate), metrics=["accuracy"])

history=model.fit(x_train, y_train,
                  batch_size=batchSize, epochs=epochs,
                  validation_data=(x_val,y_val),
                  callbacks=[es, lrr],
                 )

Epoch 1/50


In [ ]:
# plot accuracy
plt = acc_plot(history)
plt.show()

In [ ]:
# plot loss
plt = loss_plot(history)
plt.show()

In [ ]:
plt = conf_matrix(model, x_test, y_test)
plt.show()